# 『人を知る』人工知能講座 <br> <span style="color: #00B0F0;">Session 3 言語メディア</span> <br> <span style="background-color: #1F4E79; color: #FFFFFF;">&nbsp;3&nbsp;</span> BERTによる自然言語処理 〜日本語Pre-training〜 

## 1. 日本語Pre-trainedモデル

BERT日本語pre-trainedモデルを http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT%E6%97%A5%E6%9C%AC%E8%AA%9EPretrained%E3%83%A2%E3%83%87%E3%83%AB で公開しています。いくつか公開しているモデルのうち、今回は「通常版: Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip (393M; 19/11/15公開) 」を使います。zipを解凍し、/data/nlp/tool/Japanese_L-12_H-768_A-12_E-30_BPE_transformers においています。

まず、英語pre-trainedモデルと同様に、pre-trainedモデルの中身を見てみましょう。

In [1]:
!ls /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers

config.json  pytorch_model.bin	tokenizer_config.json  vocab.txt


pytorch_model.binがモデルの重みで、config.jsonは設定ファイルです。

In [2]:
!cat /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json

{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 32006
}


vocab.txtが語彙リストです。先頭20行と最後20行を見てみましょう。日本語モデルでは [unused..] を入れていません。

In [3]:
!head -n 20 /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
の
、
。
に
は
を
が
と
で
年
・
（
）
さ
して


In [4]:
!tail -n 20 /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt

アドリブ
変色
##Ｏｆｆ
##ドーレ
いつの
ＬＳＤ
アンティオキア
両性
##煎
##彬
かんする
##キングス
フィンガー
閃
論点
インディオ
スカンジナ
##紀子
１６６７
好調な


tokenizer_config.jsonは英語モデルにありませんでしたが、tokenizerのオプションを指定するものです。日本語処理する上では以下が必要です。

In [7]:
#do_lower_case：Trueにしてしまうと、濁点が消えたりとバグる
!cat /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/tokenizer_config.json

{"do_lower_case": false, "tokenize_chinese_chars": false, "init_inputs": []}


本演習ではpre-training自体は行いません。Pre-trainedモデルを使ってmasked language modelを試してみることと、プログラムの中身を見てみます。

## 2. Masked Language Modelの可視化

講義の冒頭で説明したmasked language modelは以下のプログラムで試すことができます。詳細はここでは気にしなくて結構です。

In [8]:
import copy
import torch
from transformers import BertTokenizer, BertForMaskedLM

import pandas as pd

from IPython.display import HTML

from pyknp import Juman

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Segmentation(object):
    """ Juman++でセグメンテーション """
    def __init__(self):
        self.jumanpp = Juman()
        
    def segmentation(self, text):
        result = self.jumanpp.analysis(text)
        text = " ".join([ mrph.midasi for mrph in result.mrph_list() ])
        return text
    
class MaskedLM(object):
    def __init__(self, bert_model_dir, 
                           topk=5):
        self.topk = topk
        self.segmentation = Segmentation()
  
        print("loading .. ", end="")
        self.tokenizer = BertTokenizer.from_pretrained(bert_model_dir, do_lower_case=False)
        self.model = BertForMaskedLM.from_pretrained(bert_model_dir)
        self.model.to(device)
        self.model.eval()
        print("done.")
    
    def get_masked_tokens_list(self, tokens, num_tokens):
        masked_tokens_list = []
        
        for i, token in enumerate(tokens):
            if i == 0 or i == num_tokens - 1:
                continue
            new_tokens = copy.deepcopy(tokens)
            new_tokens[i] = '[MASK]'

            masked_tokens_list.append(new_tokens)
            
        return masked_tokens_list
            
    def get_predictions(self, sentence):
        print("{}".format(sentence))
        text = self.segmentation.segmentation(sentence)
        tokens = self.tokenizer.tokenize(text)
        
        # 先頭に[CLS]、末尾に[SEP]トークンを追加
        tokens = ["[CLS]"] + tokens + ["[SEP]"]
        num_tokens = len(tokens)
        
        # 各単語をMASKする
        masked_tokens_list = self.get_masked_tokens_list(tokens, num_tokens)

        tokens_tensor = torch.tensor([ self.tokenizer.convert_tokens_to_ids(masked_tokens) for masked_tokens in masked_tokens_list ])
        tokens_tensor = tokens_tensor.to(device)
        
        # ここがメイン
        # outputs[0]に各入力トークンにおける各単語の予測確率が入っている
        # shape: (batch_size, sequence_length, config.vocab_size)
        outputs = self.model(tokens_tensor)
        predictions = outputs[0]
        
        # 予測確率が高い順にソート
        _, indices = torch.sort(predictions, descending=True)

        prediction_results = []
        inputs = []

        for i, token in enumerate(tokens):
            if i == 0 or i == num_tokens - 1:
                continue
            prediction_string = self.get_prediction_string(indices[i - 1, i, :self.topk].cpu().numpy(), token)
            inputs.append(token) 
            prediction_results.append(prediction_string)

        df = pd.DataFrame({'input': inputs, 'prediction': prediction_results})
        # indexを1始まりにする
        df.index = df.index + 1
        return df.style.set_table_styles(
                [{'selector': 'th',
                  'props': [('text-align', 'center')]}, 
                 {'selector': 'td',
                  'props': [('text-align', 'left')]}]).render()

    def get_prediction_string(self, topk_predictions, token):
        strings = []
        for predicted_token in self.tokenizer.convert_ids_to_tokens(topk_predictions):
            # 入力と一致
            if token == predicted_token:
                strings.append("<font color='red'>{}</font>".format(predicted_token))
            else:
                strings.append(predicted_token)

        return ", ".join(strings)

masked_lm = MaskedLM('/data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/')

loading .. done.


「loading .. done.」と出たら以下を実行してみましょう。

### 練習問題 1
* 好きな文を入れて、結果を考察してみましょう。

## 3. プログラムの中身の理解

プログラムの中身を少し理解してみましょう。transformers/transformers/modeling_bert.py に様々なモデルが用意されています。

このうち、BertForSequenceClassificationクラスが1文または文ペア分類タスクが用いられていたものです。(講義スライドの92ページ目の図を参照してください)

*   青色で囲った部分がメインです。入力のトークン列をベクトル列に変換します。この関数の中で、講義で説明したself-attention(12層)が動いています。(ここは完全にブラックボックスです)
*   緑色の部分で[CLS]に対応するベクトルを取得しています。(=outputs[1])
*   赤色の部分は線形の行列を作用し、緑色のベクトル(768次元)からクラス数の次元のベクトルに変換しています。
*   その後、クロスエントロピーロスを計算しています。

青色の部分をブラックボックスにしているとはいえ、これだけです。タスクによって異なるのは入力のトークン列と最終層(赤色の部分)、ロス関数だけです。